# Connecting the MCP Chatbot to Reference Servers

#### Chatbot Implementation

- Anthropic MCP Chatbot

In [1]:
%%writefile mcp_project/anthropic_mcp_chatbot_reference_server.py
from dotenv import load_dotenv
from anthropic import Anthropic
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from typing import List, Dict, TypedDict
from contextlib import AsyncExitStack
import json
import asyncio

load_dotenv()


class ToolDefinition(TypedDict):
    name: str
    description: str
    input_schema: dict


class MCP_ChatBot:
    def __init__(self):
        # Initialize session and client objects
        self.sessions: List[ClientSession] = []  # new
        self.exit_stack = AsyncExitStack()  # new
        self.anthropic = Anthropic()
        self.available_tools: List[ToolDefinition] = []  # new
        self.tool_to_session: Dict[str, ClientSession] = {}  # new

    async def connect_to_server(self, server_name: str, server_config: dict) -> None:
        """Connect to a single MCP server."""
        try:
            server_params = StdioServerParameters(**server_config)
            stdio_transport = await self.exit_stack.enter_async_context(
                stdio_client(server_params)
            )  # new
            read, write = stdio_transport
            session = await self.exit_stack.enter_async_context(
                ClientSession(read, write)
            )  # new
            await session.initialize()
            self.sessions.append(session)

            # List available tools for this session
            response = await session.list_tools()
            tools = response.tools
            print(f"\nConnected to {server_name} with tools:", [t.name for t in tools])

            for tool in tools:  # new
                self.tool_to_session[tool.name] = session
                self.available_tools.append(
                    {
                        "name": tool.name,
                        "description": tool.description,
                        "input_schema": tool.inputSchema,
                    }
                )
        except Exception as e:
            print(f"Failed to connect to {server_name}: {e}")

    async def connect_to_servers(self):  # new
        """Connect to all configured MCP servers."""
        try:
            with open("server_config.json", "r") as file:
                data = json.load(file)

            servers = data.get("mcpServers", {})

            for server_name, server_config in servers.items():
                await self.connect_to_server(server_name, server_config)
        except Exception as e:
            print(f"Error loading server configuration: {e}")
            raise

    async def process_query(self, query):
        messages = [{"role": "user", "content": query}]
        response = self.anthropic.messages.create(
            max_tokens=2024,
            model="claude-3-7-sonnet-20250219",
            tools=self.available_tools,
            messages=messages,
        )
        process_query = True
        while process_query:
            assistant_content = []
            for content in response.content:
                if content.type == "text":
                    print(content.text)
                    assistant_content.append(content)
                    if len(response.content) == 1:
                        process_query = False
                elif content.type == "tool_use":
                    assistant_content.append(content)
                    messages.append({"role": "assistant", "content": assistant_content})
                    tool_id = content.id
                    tool_args = content.input
                    tool_name = content.name

                    print(f"Calling tool {tool_name} with args {tool_args}")

                    # Call a tool
                    session = self.tool_to_session[tool_name]  # new
                    result = await session.call_tool(tool_name, arguments=tool_args)
                    messages.append(
                        {
                            "role": "user",
                            "content": [
                                {
                                    "type": "tool_result",
                                    "tool_use_id": tool_id,
                                    "content": result.content,
                                }
                            ],
                        }
                    )
                    response = self.anthropic.messages.create(
                        max_tokens=2024,
                        model="claude-3-7-sonnet-20250219",
                        tools=self.available_tools,
                        messages=messages,
                    )

                    if (
                        len(response.content) == 1
                        and response.content[0].type == "text"
                    ):
                        print(response.content[0].text)
                        process_query = False

    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")

        while True:
            try:
                query = input("\nQuery: ").strip()

                if query.lower() == "quit":
                    break

                await self.process_query(query)
                print("\n")

            except Exception as e:
                print(f"\nError: {str(e)}")

    async def cleanup(self):  # new
        """Cleanly close all resources using AsyncExitStack."""
        await self.exit_stack.aclose()


async def main():
    chatbot = MCP_ChatBot()
    try:
        # the mcp clients and sessions are not initialized using "with"
        # like in the previous lesson
        # so the cleanup should be manually handled
        await chatbot.connect_to_servers()  # new!
        await chatbot.chat_loop()
    finally:
        await chatbot.cleanup()  # new!


if __name__ == "__main__":
    asyncio.run(main())

Writing mcp_project/anthropic_mcp_chatbot_reference_server.py


- OpenAI MCP Chatbot

In [9]:
%%writefile mcp_project/openai_mcp_chatbot_reference_server.py
from dotenv import load_dotenv
from openai import OpenAI
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client
from typing import List, Dict, TypedDict
from contextlib import AsyncExitStack
import json
import asyncio

load_dotenv()


class ToolDefinition(TypedDict):
    name: str
    description: str
    input_schema: dict


class MCP_ChatBot:
    def __init__(self):
        # Initialize session and client objects
        self.sessions: List[ClientSession] = []  # new
        self.exit_stack = AsyncExitStack()  # new
        self.openai = OpenAI()
        self.available_tools: List[ToolDefinition] = []  # new
        self.tool_to_session: Dict[str, ClientSession] = {}  # new

    async def connect_to_server(self, server_name: str, server_config: dict) -> None:
        """Connect to a single MCP server."""
        try:
            server_params = StdioServerParameters(**server_config)
            stdio_transport = await self.exit_stack.enter_async_context(
                stdio_client(server_params)
            )  # new
            read, write = stdio_transport
            session = await self.exit_stack.enter_async_context(
                ClientSession(read, write)
            )  # new
            await session.initialize()
            self.sessions.append(session)

            # List available tools for this session
            response = await session.list_tools()
            tools = response.tools
            print(f"\nConnected to {server_name} with tools:", [t.name for t in tools])

            for tool in tools:  # new
                self.tool_to_session[tool.name] = session
                self.available_tools.append(
                    {
                        "type": "function",
                        "name": tool.name,
                        "description": tool.description,
                        "parameters": tool.inputSchema,
                    }
                )

                for tool in self.available_tools: 
                    tool["parameters"]["additionalProperties"] = False
        except Exception as e:
            print(f"Failed to connect to {server_name}: {e}")

    async def connect_to_servers(self):  # new
        """Connect to all configured MCP servers."""
        try:
            with open("server_config.json", "r") as file:
                data = json.load(file)

            servers = data.get("mcpServers", {})

            for server_name, server_config in servers.items():
                await self.connect_to_server(server_name, server_config)
        except Exception as e:
            print(f"Error loading server configuration: {e}")
            raise

    async def process_query(self, query: str):
        messages = [{"role": "user", "content": query}]

        is_loop_nedded = True
        while is_loop_nedded:
            response = self.openai.responses.create(
                model="gpt-4o",
                input=messages,
                tools=self.available_tools,
            )
            for block in response.output:
                if block.type == "message":
                    is_loop_nedded = False
                    return block.content[0].text

                elif block.type == "function_call":
                    tool_call_id = block.call_id
                    tool_name = block.name
                    tool_args = json.loads(block.arguments)

                    print(
                        f"Processing tool call: {tool_name} with args: {tool_args} with call_id: {tool_call_id}"
                    )

                    # Call a tool
                    session = self.tool_to_session[tool_name]  # new
                    result = await session.call_tool(tool_name, arguments=tool_args)
                    messages.append(block.model_dump())
                    messages.append(
                        {
                            "type": "function_call_output",
                            "call_id": tool_call_id,
                            "output": str(result),
                        }
                    )
                    is_loop_nedded = True

    async def chat_loop(self):
        """Run an interactive chat loop"""
        print("\nMCP Chatbot Started!")
        print("Type your queries or 'quit' to exit.")

        while True:
            try:
                query = input("\nQuery :[Type 'quit' to exit]: ").strip()

                if query.lower() == "quit":
                    break

                answer = await self.process_query(query)
                print("{}\n". format(answer))

            except Exception as e:
                print(f"\nError: {str(e)}")

    async def cleanup(self):  # new
        """Cleanly close all resources using AsyncExitStack."""
        await self.exit_stack.aclose()


async def main():
    chatbot = MCP_ChatBot()
    try:
        # the mcp clients and sessions are not initialized using "with"
        # like in the previous lesson
        # so the cleanup should be manually handled
        await chatbot.connect_to_servers()  # new!
        await chatbot.chat_loop()
    finally:
        await chatbot.cleanup()  # new!


if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_project/openai_mcp_chatbot_reference_server.py


# To Invoke the MCP Servers as Reference Servers

Below we have mentioned the 3 reference servers that we will be connecting to:
- **filesystems**: This server provides access to various file systems (i.e., local) for reading and writing files.
- **research_papers**: This server allows querying and retrieving research papers.
- **fetch**: This server is used to fetch data from external sources like APIs.

In [7]:
%%writefile mcp_project/server_config.json
{
    "mcpServers": {
        "filesystem": {
            "command": "npx",
            "args": [
                "-y",
                "@modelcontextprotocol/server-filesystem",
                "."
            ]
        },
        "research": {
            "command": "uv",
            "args": [
                "run",
                "research_server.py"
            ]
        },
        "fetch": {
            "command": "uvx",
            "args": [
                "mcp-server-fetch"
            ]
        }
    }
}

Overwriting mcp_project/server_config.json


### How to Run the MCP Chatbot using the Reference Servers

Steps to run the MCP Chatbot using the reference servers:
1. **OpenYour Terminal**: Open your terminal or command prompt.
2. **Navigate to the Directory**: Change to the directory where your MCP notebook is located as we stored within the `mcp_project` folder.
   ```bash
   cd /path/to/mcp-notebooks/cookbooks/mcp_project
   ```
3. **Create a Virtual Environments using UV**: 
   ```bash
   uv init
   ```
4. **Install Required Packages**: Install the required packages using the following command:
   ```bash
   uv add anthropic mcp fastmcp openai arxiv python-dotenv nest-asyncio
   ```

5. **Run the MCP Notebook**: Start the MCP notebook using the following command:
   ```bash
   uv run openai_mcp_chatbot_reference_server.py
   ```

Finally, you can interact with the MCP Chatbot in your terminal. You can ask questions, and the chatbot will respond using the connected reference servers.

#### The Terminal Output

<img src="assets/mcp_references_server_connect_via_mcp_client_chatbot.png" width="auto" alt="MCP Inspector - Search Papers Tools">

#### The Output of the MCP Chatbot for The MCP Mermaid Diagram using Code
```mermaid
graph TD;
    A[MCP Client] -->|Communicates with| B[MCP Server];
    A --> C[User Interface];
    A --> D[Local Database];
    B --> E[Core Logic];
    B --> F[External API];
    E --> G[Data Processing];
    E --> H[Business Logic];
    F --> I[Third-party Service];
```